In [2]:
import requests
import pandas as pd
from datetime import datetime, timedelta


In [4]:
def generate_weather_api_urls(api_key, location, years_back=3):
    base_url = "http://api.worldweatheronline.com/premium/v1/past-weather.ashx"
    urls = []

    # Pobranie biezacej daty
    current_year = datetime.now().year

    # Pętla z ostatnimi 4 latami
    for year in range(current_year - years_back, current_year+1):
        for month in range(1, 4):  # zakres od stycznia do marca
            # definiowanie dnia startowego i koncowego dla kazdego miesiaca
            start_date = datetime(year, month, 1)
            if month == 12:
                end_date = datetime(year + 1, 1, 1) - timedelta(days=1)
            else:
                end_date = datetime(year, month + 1, 1) - timedelta(days=1)
            
            # Formatowanie daty do 'YYYY-MM-DD'
            start_date_str = start_date.strftime('%Y-%m-%d')
            end_date_str = end_date.strftime('%Y-%m-%d')
            
            # Tworzenie API
            url = f"{base_url}?key={api_key}&q={location}&format=json&date={start_date_str}&enddate={end_date_str}&tp=24"
            urls.append(url)

    return urls

In [5]:
def extract_weather_data(weather_data):
    extracted_data = []
    
    for day in weather_data:
        date = day["date"]
        maxtempC = day["maxtempC"]
        mintempC = day["mintempC"]
        avgtempC = day["avgtempC"]
        sunHour = day["sunHour"]
        humidity = day["hourly"][0]["humidity"]
        pressure = day["hourly"][0]["pressure"]
        
        extracted_data.append({
            "date": date,
            "maxtempC": maxtempC,
            "mintempC": mintempC,
            "avgtempC": avgtempC,
            "sunHour": sunHour,
            "humidity": humidity,
            "pressure": pressure
        })
    
    return extracted_data

# Nasz klucz oraz wybrana lokalizacja
api_key = "4597174352ee463084a193409241305"
location = "London"
urls = generate_weather_api_urls(api_key, location)

all_weather_data = []

for url in urls:
    response = requests.get(url)
    data = response.json()
    weather_data = data["data"]["weather"]
    month_weather_data = extract_weather_data(weather_data)
    all_weather_data.extend(month_weather_data)


df = pd.DataFrame(all_weather_data)
df

,date,maxtempC,mintempC,avgtempC,sunHour,humidity,pressure
0,2021-01-01,3,-1,1,4.0,86,1012
1,2021-01-02,4,0,3,1.0,85,1017
2,2021-01-03,5,0,3,4.0,84,1018
3,2021-01-04,4,2,3,1.0,80,1020
4,2021-01-05,3,1,3,1.0,91,1021
...,...,...,...,...,...,...,...
356,2024-03-27,10,5,7,4.0,83,982
357,2024-03-28,10,5,7,5.0,79,980
358,2024-03-29,12,6,8,5.0,84,987
359,2024-03-30,13,7,10,5.0,75,995


In [8]:
df[50:70]


,date,maxtempC,mintempC,avgtempC,sunHour,humidity,pressure
50,2021-02-20,15,10,12,8.0,72,1004
51,2021-02-21,14,9,11,7.0,77,1008
52,2021-02-22,13,8,10,5.0,77,1014
53,2021-02-23,12,7,10,8.0,71,1023
54,2021-02-24,14,11,12,11.0,68,1023
55,2021-02-25,11,7,10,1.0,73,1026
56,2021-02-26,13,4,8,11.0,59,1037
57,2021-02-27,13,4,8,11.0,64,1040
58,2021-02-28,10,4,7,11.0,71,1037
59,2021-03-01,10,5,7,11.0,74,1031
